# WB RecSys Project

# Общее описание проекта

Необходимо на основании взаимодействий пользователей с товарами предсказать следующие взаимодействия пользователей с товарами.

# Stage 3

- Сформировать обучающую выборку
- Спроектировать схему валидации с учетом специфики задачи
- Обосновать выбор способа валидации


# Preprocessing train_data

# Импорт библиотек

In [1]:
import numpy as np
import polars as pl
import dill
from datetime import datetime, timedelta

import seaborn as sns
import matplotlib.pyplot as plt

from IPython.display import Image

import warnings

warnings.filterwarnings("ignore")

### Путь до данных

In [2]:
data_path = "../../data/closed/"
data_load_path = "../../data/load/"

# train data

### Чтение 

In [ ]:
pl.scan_parquet(data_load_path + "train_data_10_10_24_10_11_24_final.parquet").schema

In [ ]:
interactions_df = (
    pl.scan_parquet(data_load_path + "train_data_10_10_24_10_11_24_final.parquet")
    # Отбираем необходимые колонки
    .select(["wbuser_id", "nm_id", "dt"])
    # Отсортируем по дате
    .sort(by="dt")
    # Для удобства переименуем колонки
    .rename(
        {
            "wbuser_id": "user_id",
            "nm_id": "item_id",
        }
    )
    # Выполняем
    .collect()
)
interactions_df

In [ ]:
min_date = interactions_df["dt"].min()
max_date = interactions_df["dt"].max()

print(f'min_date = {min_date.strftime("%Y-%m-%d %H:%M:%S")}')
print(f'max_date = {max_date.strftime("%Y-%m-%d %H:%M:%S")}')

In [6]:
# # Посмотрим распределение покупок по часам для каждого из дней

# min_date = interactions_df["dt"].min()
# max_date = interactions_df["dt"].max()

# n_days = (max_date - min_date).days + 1

# fig, ax = plt.subplots(1, n_days, figsize=(18, 6), sharey=True)

# for i_day in range(n_days):

#     cur_day = min_date + timedelta(days=i_day)
#     next_day = cur_day + timedelta(days=1)

#     cur_day = datetime(year=cur_day.year, month=cur_day.month, day=cur_day.day)
#     next_day = datetime(year=next_day.year, month=next_day.month, day=next_day.day)

#     days = (
#         interactions_df["dt"]
#         .filter(interactions_df["dt"].is_between(cur_day, next_day))
#         .value_counts()
#         .group_by_dynamic("dt", every="1h")
#         .agg(pl.col("count").sum())
#         .sort(by="dt")
#     )

#     sns.barplot(
#         x=days["dt"].dt.hour(),
#         y=days["count"],
#         ax=ax[i_day],
#     )

#     ax[i_day].set_title(f"Распределение заказов {cur_day.strftime('%Y-%m-%d')}")
#     ax[i_day].set_xlabel("Время, часы")
#     ax[i_day].set_ylabel("Число заказов")


# plt.show()

## 1. Разделим данные на train \ test

In [7]:
# Конечная дата
max_date = interactions_df["dt"].max()

# Дата начала данных для теста
train_test_sep = max_date - timedelta(hours=8)

# Данные для теста
test_df = interactions_df.filter(interactions_df["dt"] >= train_test_sep)

# Данные для обучения моделей первого 
# и второго уровня (разделение будет потом)
train_df = interactions_df.filter(interactions_df["dt"] < train_test_sep)

Сохраним в parquet

In [8]:
train_df.write_parquet(data_path + "train_df.parquet")

### Соберем test_df

Соберем следующие просмотренные товары в списки

In [9]:
# Таблица взаимодействий уже была заранее отсортирована по дате,
# так что порядок взаимодействий по дате сохранится 
test_df = test_df.group_by("user_id").agg(pl.col("item_id")).sort(by="user_id")

Теперь следующий вопрос: сколько товаров рекомендовать? 

In [ ]:
# Информации о длинах интеракций пользователей в test_df
test_df["item_id"].map_elements(len).describe()

Составим отранжированный список товаров 

> по сути необходимо только для замера метрик, а для продакшена необходимы будут только айдишники пользователей

In [11]:
def format_item_id_test_df(x: pl.List):
    """Formats item IDs based on frequency within a user's viewing history using Polars."""
    if len(x) > 1:
        return x.value_counts().sort("count", descending=True)[""]
    return x

In [12]:
test_df = test_df.with_columns(pl.col("item_id").map_elements(format_item_id_test_df))

Сохраним в parquet

In [13]:
test_df.write_parquet(data_path + "test_df.parquet")

## 2. Модифицируем только таблицу train_df


In [14]:
train_df_weights = (
    pl.scan_parquet(data_path + "train_df.parquet").group_by(["user_id", "item_id"])
    # Посчитаем количество взаимодействий пользователя
    # с каждым конкретным товаром
    .agg(pl.col("item_id").count().alias("ui_inter"))
)

train_df_weights = train_df_weights.join(
    train_df_weights.select(["user_id", "ui_inter"])
    .group_by("user_id")
    .agg(pl.col("ui_inter").sum().alias("u_total_inter")),
    on="user_id",
    how="left",
).with_columns((pl.col("ui_inter") / pl.col("u_total_inter")).alias("weight"))

train_df_weights.collect().write_parquet(data_path + "train_df_weights.parquet")

Посчитаем количество взаимодействий с определенным товаром (`item_count`)
на его основе расчитаем рейтинг товара (`item_rating`)


In [ ]:
# Все взаимодействия с каждым товаром
item_rating_df = (
    pl.scan_parquet(data_path + "train_df_weights.parquet")
    .select(["item_id", "ui_inter"])
    .group_by("item_id")
    .agg(pl.col("ui_inter").sum().alias("item_count"))
    .collect()
)
# Общий вес\рейтинг товара по всем пользователям
item_rating_df = item_rating_df.with_columns(
    (pl.col("item_count") / item_rating_df.shape[0]).alias("item_rating")
).sort(by="item_rating", descending=True)

item_rating_df

Сохраним таблицы

In [16]:
item_rating_df.write_parquet(data_path + "item_rating_df.parquet")

## 3. Мерджим таблицу с весами к основной

Теперь будем присоединять веса к общей таблице (где существуют данные о дате взаимодействия)

In [17]:
train_df = pl.scan_parquet(data_path + "train_df.parquet")
train_df_weights = pl.scan_parquet(data_path + "train_df_weights.parquet")

# Иерджим и сохраняем в parquet
train_df.join(
    train_df_weights,
    on=["user_id", "item_id"],
    how="left",
).collect().write_parquet(data_path + "train_df.parquet")

## 4. Добавляем новые фитчи к train_df

In [18]:
(
    pl.scan_parquet(data_path + "train_df.parquet")
    # Подсчет порядковых номеров взаимодействия пользователя с 
    # каждым конкретынм товаром
    #
    # Т.е. если взаимодействия с товарами идут в следующем порядке
    # [1, 2, 1, 1, 2, 4]
    #
    # то результат будет следующим:
    #
    # cumcount([1, 2, 1, 1, 2, 4]) + 1 = [1, 1, 2, 3, 2, 1]
    #
    # Можно расчитать быстрее, используя оконную функцию аналогично предыдущему запросу
    # Но у меня не умещается такой вариант в оперативной памяти
    .with_columns(
        (pl.int_range(1, pl.len() + 1)).over(["user_id", "item_id"]).alias("ui_entry")
    )
    # кумулятивный вес товара на момент просмотра
    .with_columns(
        (pl.col("ui_entry") / pl.col("ui_inter") * pl.col("weight")).alias("cum_weight")
    )
    # сортировать не обязательно, но хорошо будет
    # если отсортировать по времени, т.к. в дальнейшем 
    # записи будут делиться по времени и сортировка ускорит процесс:
    # predict блок в в процессоре не будет "спотыкаться"
    .sort(by=["dt"])
    .collect()
# сразу сохраним в parquet
).write_parquet(data_path + "train_df.parquet")

## 5. Разбиение train_df на base_model_data и ranker_data

In [19]:
train_df = (
    pl.scan_parquet(data_path + "train_df.parquet")
)

> Под обучение модели второго уровня выделим 1/5 от данных идущих на train, т.е. ~8 часов.

In [32]:
# Дата последней интеракиции
max_data = train_df.select("dt").max().collect().item()

# Дата разделяющая данные для трейна моделей
# первого и второго уровней
base_ranker_sep = max_data - timedelta(hours=8)

# Данные для обучения моделей первого уровня
# ranker_data = train_df[(train_df["dt"] >= base_ranker_sep)]
# Сразу сохраим в бинарник
train_df.filter(pl.col("dt") >= base_ranker_sep).collect().write_parquet(data_path + "ranker_data.parquet")

# Данные для обучения модели второго уровня
# base_models_data = train_df[(train_df["dt"] < base_ranker_sep)]
# Сразу сохраим в бинарник
train_df.filter(pl.col("dt") < base_ranker_sep).collect().write_parquet(data_path + "base_models_data.parquet")


## Выделим группы пользователей 

In [3]:
# Уникальные айдишники пользователей в таблицах

base_users = (
    pl.scan_parquet(data_path + "base_models_data.parquet")
    .select("user_id")
    .unique()
    .collect()
).to_numpy()
base_users = base_users.reshape(base_users.shape[0])
# save
with open(data_path + "base_users.dill", "wb") as f:
    dill.dump(base_users, f)

ranker_users = (
    pl.scan_parquet(data_path + "ranker_data.parquet")
    .select("user_id")
    .unique()
    .collect()
).to_numpy()
ranker_users = ranker_users.reshape(ranker_users.shape[0])
# save
with open(data_path + "ranker_users.dill", "wb") as f:
    dill.dump(ranker_users, f)

test_users = (
    pl.scan_parquet(data_path + "test_df.parquet")
    .select("user_id")
    .unique()
    .collect()
).to_numpy()
test_users = test_users.reshape(test_users.shape[0])
# save
with open(data_path + "test_users.dill", "wb") as f:
    dill.dump(test_users, f)


In [ ]:
# Пользователи, которым надо выдавать пресказания для обучения ранкера,
# т.е. присутствуют и в base_models_data и в ranker_data (base to ranker users)
b2r_users = np.array(list((set(base_users) & set(ranker_users))))
display("b2r_users", b2r_users, b2r_users.shape)
# save
with open(data_path + "b2r_users.dill", "wb") as f:
    dill.dump(b2r_users, f)


# на оставшихся пользователях ранкер обучаться не будет
# на них просто не будет скоров
ranker_only_users = np.array(list(set(ranker_users) - set(base_users)))
display("ranker_only_users", ranker_only_users, ranker_only_users.shape)
# save
with open(data_path + "ranker_only_users.dill", "wb") as f:
    dill.dump(ranker_only_users, f)

# Проверим качество на тестовой выборке
# Берем только пользователей, которые присутствуют
# в base и test выборках
b2t_users = np.array(list(set(test_users) & (set(base_users))))
display("b2t_users", b2t_users, b2t_users.shape)
with open(data_path + "b2t_users.dill", "wb") as f:
    dill.dump(b2t_users, f)

# Пользователи из test_df, которым будут выданы
# таргетирвонные рекомондации
bNr2t_users = np.array(list((set(base_users) | set(ranker_users)) & set(test_users)))
display("bNr2t_users", bNr2t_users, bNr2t_users.shape)
# save
with open(data_path + "bNr2t_users.dill", "wb") as f:
    dill.dump(bNr2t_users, f)


# Пользователи, которые присутствуют только в test_df (cold_users)
test_only_users = np.array(list(set(test_users) - (set(base_users) | set(ranker_users))))
display("test_only_users", test_only_users, test_only_users.shape)
# save
with open(data_path + "test_only_users.dill", "wb") as f:
    dill.dump(test_only_users, f)